In [1]:
import pandas as pd
import numpy as np
import json
from sklearn.metrics.pairwise import cosine_similarity
import datetime
from sklearn.preprocessing import LabelEncoder

In [2]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [3]:
%cd /content/drive/MyDrive/10273 [AI-TM29] Prepare a suitably sized data sets for regular and part number based product catalog

/content/drive/.shortcut-targets-by-id/1pWIbUZ_5hAIn8XEJd7oNpZI7SiF7aQrw/10273 [AI-TM29] Prepare a suitably sized data sets for regular and part number based product catalog


In [4]:
rev_df = pd.read_csv('User_Orders.csv')

rev_df.head()

orderNumber  orderedDate      customerID        customerName            asin
0   93f35a6b9  09 22, 2013  A1BB77SEBQT8VX  Darrow H Ankrum II  ['B00007GDFV']
1   d6ffc1262  07 17, 2013   AHWOW7D1ABO9C              rosieO  ['B00007GDFV']
2   b52b6a408  04 13, 2013   AKS3GULZE0HFC          M. Waltman  ['B00007GDFV']
3   d6eba872b   03 9, 2013   A38NS6NF6WPXS            BTDoxies  ['B00007GDFV']
4   87e6fdc25  01 27, 2013  A1KOKO3HTSAI1H        Robin Howard  ['B00007GDFV']

In [5]:
rev_df = rev_df.drop(['orderedDate', 'customerID', 'customerName'], axis=1)
rev_df

orderNumber            asin
0        93f35a6b9  ['B00007GDFV']
1        d6ffc1262  ['B00007GDFV']
2        b52b6a408  ['B00007GDFV']
3        d6eba872b  ['B00007GDFV']
4        87e6fdc25  ['B00007GDFV']
...            ...             ...
180104   afad3a759  ['B01HIX20DQ']
180105   7202904a2  ['B01HIX20DQ']
180106   6e2adc762  ['B01HIYPFXC']
180107   a9f56573f  ['B01HJ6HHD0']
180108   d1f0bddaa  ['B01HJ6HHD0']

[180109 rows x 2 columns]

In [6]:
rev_df['asin'] = rev_df['asin'].str.replace("'", "")
rev_df.iloc[60:71]

orderNumber                                  asin
60   01c7ae5db                          [B00007GDFV]
61   f737aff98                          [B00007GDFV]
62   8e5c05feb                          [B00007GDFV]
63   f7d950557  [B00007GDFV, B00B6AWXYE, B00EZMCOCQ]
64   d6b4228ed                          [B00007GDFV]
65   eca3d5363                          [B00007GDFV]
66   2faa6025b                          [B00007GDFV]
67   bd7540f73                          [B00007GDFV]
68   2cec1b58b                          [B00007GDFV]
69   38914494c                          [B00007GDFV]
70   13130aa4d                          [B00007GDFV]

In [7]:
# Remove square brackets and split the string
rev_df['asin'] = rev_df['asin'].str.strip('[]').str.split(', ')

# Explode the column
result = rev_df.explode('asin')
result.iloc[60:71]

orderNumber        asin
60   01c7ae5db  B00007GDFV
61   f737aff98  B00007GDFV
62   8e5c05feb  B00007GDFV
63   f7d950557  B00007GDFV
63   f7d950557  B00B6AWXYE
63   f7d950557  B00EZMCOCQ
64   d6b4228ed  B00007GDFV
65   eca3d5363  B00007GDFV
66   2faa6025b  B00007GDFV
67   bd7540f73  B00007GDFV
68   2cec1b58b  B00007GDFV

In [ ]:
result.to_csv('check1.csv', index=False)

In [ ]:
pivot_df = result.pivot_table(index='orderNumber', columns='asin', aggfunc='size', fill_value=0)


In [ ]:
# Calculate cosine similarity
cosine_sim = cosine_similarity(pivot_df)

In [ ]:
# Convert ASINs to numerical labels
le = LabelEncoder()
asin_labels = le.fit_transform(result['asin'])
asin_mapping = dict(zip(le.classes_, asin_labels))

In [ ]:
# Function to get product recommendations
def get_recommendations(asin, num_recommendations=3):
    asin_label = asin_mapping.get(asin)
    if asin_label is None:
        return "ASIN not found in the dataset"

    # Calculate similarity scores for the given ASIN
    sim_scores = list(enumerate(cosine_sim[asin_label]))

    # Sort the products based on similarity scores
    sim_scores = sorted(sim_scores, key=lambda x: x[1], reverse=True)

    # Get top N similar products
    top_products = sim_scores[1:num_recommendations+1]
    top_product_labels = [item[0] for item in top_products]
    top_product_asins = le.inverse_transform(top_product_labels)

    return top_product_asins

In [ ]:
# Example usage
input_asin = 'B00B6AWXYE'
recommendations = get_recommendations(input_asin)
print(f"Top 3 recommendations for {input_asin}: {recommendations}")

Top 3 recommendations for B00B6AWXYE: ['B0004HD8VW' 'B000CKHTCA' 'B000GHRZN2']
